In [1]:
import argparse
import os
import torch
import numpy as np
import cv2
import sys
import shutil
import matplotlib.pyplot as plt

from attrdict import AttrDict

from sgan.data.loaderJAAD import data_loader
from sgan.models import CNNLSTMJAAD2

parser = argparse.ArgumentParser()
parser.add_argument('--model_path', type=str)
parser.add_argument('--dset_type', default='test', type=str)

def create_folder(directory, renew=False):
    
    # if directory doesn't exist
    if not os.path.exists(directory):
        os.makedirs(directory)
    
    # if already exist, delete and re-create
    elif os.path.exists(directory) and renew:
        shutil.rmtree(directory)
        os.makedirs(directory)

def get_classifier(checkpoint):
        
    args = AttrDict(checkpoint['args'])  
    classifier = CNNLSTMJAAD2(
        embedding_dim=args.embedding_dim,
        h_dim=args.h_dim,
        dropout=0.0,
        grad=False)
    classifier.load_state_dict(checkpoint['best_state'])
    return classifier

def save_gradients(gradients, pedestrian_filenames, result):
    
    # get the filename
    pedestrian_filenames = [[f.split("/")[-4], f.split("/")[-2], f.split("/")[-1]] for f in pedestrian_filenames] 
    # create folder for pedestrian
    create_folder(os.path.join(os.getcwd(), "results", pedestrian_filenames[0][0], result, pedestrian_filenames[0][1]))
    
    for g,f in zip(gradients, pedestrian_filenames):
        print(f, result)
        print(np.amax(g), np.amin(g))
        cv2.imwrite(os.path.join(os.getcwd(), "results", f[0], result, f[1], f[2]), np.uint8(g*255))

def save_classification(pedestrian_images, decision, pedestrian_filenames, result):
    
    pedestrian_images = np.transpose(pedestrian_images, axes=(0,2,3,1))
        
    # get the filename
    pedestrian_filenames = [[f.split("/")[-4], f.split("/")[-2], f.split("/")[-1]] for f in pedestrian_filenames] 
    # create folder for pedestrian
    create_folder(os.path.join(os.getcwd(), "results", pedestrian_filenames[0][0], result, pedestrian_filenames[0][1]))
    
    for i,f,d in zip(pedestrian_images, pedestrian_filenames,decision):
        print(f, d, result)
        cv2.imwrite(os.path.join(os.getcwd(), "results", f[0], result, f[1], f[2][0:-4]+"-"+str(d)+".png"), np.uint8(i*255))
        
def guidedbackprop(args, loader, classifier):
       
    classifier.train()        
    for batch in loader:

        (pedestrian_crops, decision_true, pedestrian_filenames) = batch
                    
        # predict decision
        decision_pred_scores = classifier(pedestrian_crops, input_as_var=True)        
        if(torch.cuda.is_available()):
            decision_onehot = torch.cuda.FloatTensor(1, decision_pred_scores.size()[-1]).zero_()
        else:
            decision_onehot = torch.FloatTensor(1, decision_pred_scores.size()[-1]).zero_()
        decision_onehot[0][decision_pred_scores.max(1)[1]] = 1
        if(torch.cuda.is_available()):
            decision_onehot = decision_onehot.cuda()
        else:
            decision_onehot = decision_onehot
                 
        # backprop
        classifier.zero_grad()
        decision_pred_scores.backward(gradient=decision_onehot)
        gradients = classifier.gradients.cpu().numpy()
               
        # normalize each image
        gradients_min = np.amin(np.reshape(gradients, newshape=(np.shape(gradients)[0],-1)), 1)
        gradients_max = np.amax(np.reshape(gradients, newshape=(np.shape(gradients)[0],-1)), 1)
        gradients -= gradients_min[:,None,None,None]
        gradients /= gradients_max[:,None,None,None] - gradients_min[:,None,None,None]
        gradients = np.transpose(gradients, (0, 2, 3, 1))
                
        # save images to folder
        decision_true = decision_true.item()
        decision_pred = decision_pred_scores.max(1)[1].item()
        
        # tn
        if(decision_pred == 0 and decision_true == 0):
            save_gradients(gradients, pedestrian_filenames[0], "tn")            
        # fn
        if(decision_pred == 0 and decision_true == 1):
            save_gradients(gradients, pedestrian_filenames[0], "fn")            
        # fp
        if(decision_pred == 1 and decision_true == 0):
            save_gradients(gradients, pedestrian_filenames[0], "fp")            
        # tp
        if(decision_pred == 1 and decision_true == 1):
            save_gradients(gradients, pedestrian_filenames[0], "tp")

# only works for batch size of 1
def classify_every_timestep(args, loader, classifier):
       
    # initialize results dict
    images_dict = dict()
    standing_dict = dict()
    looking_dict = dict()
    walking_dict = dict()
    crossing_dict = dict()
        
    classifier.eval()        
    for batch in loader:

        # get batch
        (pedestrian_crops, standing_true, looking_true, walking_true, crossing_true, pedestrian_foldernames, pedestrian_filenames) = batch
        
        # predict decision
        standing_pred, looking_pred, walking_pred, crossing_pred = classifier(pedestrian_crops,classify_every_timestep=True)
        
        _, standing_pred = standing_pred[0].max(1)
        _, looking_pred = looking_pred[0].max(1)
        _, walking_pred = walking_pred[0].max(1)
        _, crossing_pred = crossing_pred[0].max(1)
        standing_pred = standing_pred.numpy()
        looking_pred = looking_pred.numpy()
        walking_pred = walking_pred.numpy()
        crossing_pred = crossing_pred.numpy()
                
        # update dictionaries
        images_dict[str(pedestrian_foldernames[0][0])] = [str(pedestrian_filenames[0][0])]                
        standing_dict[str(pedestrian_foldernames[0][0])] = [standing_pred[0]]
        looking_dict[str(pedestrian_foldernames[0][0])] = [looking_pred[0]]
        walking_dict[str(pedestrian_foldernames[0][0])] = [walking_pred[0]]
        crossing_dict[str(pedestrian_foldernames[0][0])] = [crossing_pred[0]]
        for i in range(1,len(crossing_pred)):
            images_dict[str(pedestrian_foldernames[0][0])].append(str(pedestrian_filenames[0][i]))
            standing_dict[str(pedestrian_foldernames[0][0])].append(standing_pred[i])
            looking_dict[str(pedestrian_foldernames[0][0])].append(looking_pred[i])
            walking_dict[str(pedestrian_foldernames[0][0])].append(walking_pred[i])
            crossing_dict[str(pedestrian_foldernames[0][0])].append(crossing_pred[i])
        
    return images_dict, standing_dict, looking_dict, walking_dict, crossing_dict
                
def main(args):
            
    # create results folder if it does not exist
    # - used for guidedbackprop
    results_root = os.path.join(os.getcwd(), "results")
    create_folder(results_root, renew=False)
    
    # model path
    path = args.model_path
    print("model path ", path)

     # get the model
    checkpoint = torch.load(path, map_location='cpu')
    classifier = get_classifier(checkpoint)

    # generate the arguments
    _args = AttrDict(checkpoint['args']) 
    _args.batch_size = 1
    print(_args)
        
    # get the dataset
    val_path = os.path.join(args.dataset, "val") 
    dset, loader = data_loader(_args, val_path, "val")
    
    #print('Running Guided Backprop')
    #guidedbackprop(_args, loader, classifier)

    print('Classifying at every timestep')
    images_dict, standing_dict, looking_dict, walking_dict, crossing_dict = classify_every_timestep(_args, loader, classifier)
    return images_dict, standing_dict, looking_dict, walking_dict, crossing_dict
        
if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument('--model_path', type=str)
    parser.add_argument('--dset_type', default='test', type=str)

    parser.model_path = "models/vgg16-jaad.pt" 
    parser.dataset = "./datasets/jaad"
    args = parser
    images_dict, standing_dict, looking_dict, walking_dict, crossing_dict = main(args)

model path  models/vgg16-jaad.pt
AttrDict({'print_every': 100, 'max_obs_len': 10, 'restore_from_checkpoint': 1, 'dropout': 0.8, 'min_obs_len': 8, 'timing': 0, 'use_gpu': 1, 'dataset': './datasets/jaad', 'learning_rate': 0.001, 'batch_norm': False, 'batch_size': 1, 'checkpoint_every': 100, 'timestep': 10, 'gpu_num': '0', 'num_layers': 1, 'mlp_dim': -1, 'h_dim': 32, 'checkpoint_start_from': 'None', 'output_dir': './models', 'num_iterations': 2500, 'num_epochs': 200, 'loader_num_workers': 1, 'checkpoint_name': 'vgg16-jaad', 'embedding_dim': -1})
Reading annotations from  ./datasets/jaad\val


100%|██████████████████████████████████████████████████████████████████████████████████| 91/91 [00:08<00:00,  7.42it/s]

 Processing sequence



 96%|███████████████████████████████████████████████████████████████████████████▊   | 120/125 [00:00<00:00, 217.73it/s]

 Processing sequence



100%|████████████████████████████████████████████████████████████████████████████████| 125/125 [00:03<00:00, 36.00it/s]

no crossers:  19
no noncrossers:  34
Classifying at every timestep


100%|████████████████████████████████████████████████████████████████████████████████| 125/125 [00:20<00:00, 36.00it/s]

In [ ]:
videonames = ["video_"+str(x).zfill(4)+".txt" for x in range(301,346+1)]

video_no = [str(x).zfill(4) for x in range(301,346)]

print(videonames)
#print(images_dict)

for key, value in images_dict.items():
    #print(key.split("/")[1])    
    if(key.split("/")[1] == video_no[0]):
        print(key, video_no[0])

#import pandas as pd
#df = pd.read_csv("./datasets/lausanne/val/annotations/Ouchy-2-Left-final.txt")


In [2]:
# folderpath1 and folderpath2 are the sequences(pedestrians) that have been classified
# filename is the image i.e. frame number
# prediction is the predicted decision
# - iterate through the list of folderpaths
# - loop through the dataframe contents
# - set the prediction
import pandas as pd
video_no = [str(x).zfill(4) for x in range(301,346)]

for v in video_no:
    
    df = pd.read_csv("./datasets/jaad/val/annotations/video_"+v+".txt")
    
    # images_dict, standing_dict, looking_dict, walking_dict, crossing_dict
    for count, ((folderpath1, filename), 
                (_, standing), 
                (_, looking), 
                (_, walking),
                (_, crossing)) in enumerate(zip(images_dict.items(), 
                                                standing_dict.items(), 
                                                looking_dict.items(), 
                                                walking_dict.items(),
                                                crossing_dict.items())):

        if(folderpath1.split("/")[1] != v):
            continue
        
        print(folderpath1, filename)
        
        # get the filenames (images) in the dataframe for the current pedestrian
        _standing = -1
        _looking = -1
        _walking = -1
        _crossing = -1
        for f in df.loc[(df["folderpath"] == folderpath1)]["filename"]:
            # if the image has been given a label by the classifier
            if(f in filename):
                # index at which the filename occurs
                i = filename.index(f)
                
                # get the classification given the index
                _standing = standing[i]
                _looking = looking[i]
                _walking = walking[i]
                _crossing = crossing[i]
                                
                df.loc[(df["folderpath"] == folderpath1) & (df["filename"] == f), "standing_pred"] = _standing
                df.loc[(df["folderpath"] == folderpath1) & (df["filename"] == f), "looking_pred"] = _looking
                df.loc[(df["folderpath"] == folderpath1) & (df["filename"] == f), "walking_pred"] = _walking
                df.loc[(df["folderpath"] == folderpath1) & (df["filename"] == f), "crossing_pred"] = _crossing
            else:
                # set it to the previous decision if it is not in the list of predicted images
                df.loc[(df["folderpath"] == folderpath1) & (df["filename"] == f), "standing_pred"] = _standing
                df.loc[(df["folderpath"] == folderpath1) & (df["filename"] == f), "looking_pred"] = _looking
                df.loc[(df["folderpath"] == folderpath1) & (df["filename"] == f), "walking_pred"] = _walking
                df.loc[(df["folderpath"] == folderpath1) & (df["filename"] == f), "crossing_pred"] = _crossing
   
    df.to_csv("./datasets/jaad/val/annotations/final_"+v+".txt", sep=',', encoding='utf-8')   

crops/0301/pedestrian1 ['0016.png', '0026.png', '0036.png', '0046.png', '0056.png', '0066.png', '0076.png', '0086.png', '0096.png', '0106.png']
crops/0301/pedestrian2 ['0053.png', '0063.png', '0073.png', '0083.png', '0093.png', '0103.png', '0113.png', '0123.png', '0133.png', '0143.png']
crops/0302/pedestrian4 ['0004.png', '0014.png', '0024.png', '0034.png', '0044.png', '0054.png', '0064.png', '0074.png', '0084.png']
crops/0303/pedestrian1 ['0002.png', '0012.png', '0022.png', '0032.png', '0042.png', '0052.png', '0062.png', '0072.png', '0082.png']
crops/0304/pedestrian ['0004.png', '0014.png', '0024.png', '0034.png', '0044.png', '0054.png', '0064.png', '0074.png', '0084.png']
crops/0305/pedestrian6 ['0060.png', '0070.png', '0080.png', '0090.png', '0100.png', '0110.png', '0120.png', '0130.png', '0140.png', '0150.png']
crops/0305/pedestrian7 ['0060.png', '0070.png', '0080.png', '0090.png', '0100.png', '0110.png', '0120.png', '0130.png', '0140.png', '0150.png']
crops/0305/pedestrian8 ['0060

In [ ]:
in_videos = ["./datasets/jaad/val/videos/video_"+x+".mp4" for x in range(301,346+1)]
op_videos = ["./datasets/jaad/val/videos/final_"+x+".mp4" for x in range(301,346+1)]
annotations = ["./datasets/jaad/val/annotations/final_"+x+".txt" for x in range(301,346+1)]

In [3]:
import os
import sys
import cv2
import glob
import shutil
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from tqdm import tqdm

in_videos = ["./datasets/jaad/val/videos/video_"+str(x).zfill(4)+".mp4" for x in range(301,346+1)]
op_videos = ["./datasets/jaad/val/videos/final_"+str(x).zfill(4)+".mp4" for x in range(301,346+1)]
annotations = ["./datasets/jaad/val/annotations/final_"+str(x).zfill(4)+".txt" for x in range(301,346+1)]

for in_video,op_video,annotation in zip(in_videos,op_videos,annotations):

    fontface = cv2.FONT_HERSHEY_SIMPLEX;
    scale = 0.4;
    thickness = 1;
    baseline = 0;
    # ------- args ------- #

    # ------- utils ------- #

    # ------- utils ------- #

    # ------- main ------- #
    df = pd.read_csv(annotation)

    in_cap = cv2.VideoCapture(in_video)
    op_cap = cv2.VideoWriter(op_video, cv2.VideoWriter_fourcc(*'XVID'), 30, (int(in_cap.get(3)),int(in_cap.get(4))))

    print("Drawing trajectories")
    pbar = tqdm(total = df["frame"].max())
    t = 1
    while(in_cap.isOpened()):

        pbar.update(1)

        # get frame
        ret, im1 = in_cap.read()
        im = np.copy(im1)
        if(ret==False):
            break

        for row in df[df["frame"] == t].itertuples(index=True, name='Pandas'):        
            #if((row.height <= 50) or (row.tly + row.height + 40 > np.shape(im)[0])):
            # red box for crossing
            if(row.type == "pedestrian"):# and row.x > min_lifetime_crossers):

                # put ground truth
                label = "true: " + str(row.crossing_true)
                text = cv2.getTextSize(label, fontface, scale, thickness);
                cv2.rectangle(im, (row.x, row.y-20), (row.x+row.w, row.y-10), (0,0,0), -1);
                cv2.putText(im, label, (row.x, row.y-10), fontface, scale, (255,255,255), thickness, 8);

                # put prediction
                label = "pred: " + str(row.crossing_pred)
                text = cv2.getTextSize(label, fontface, scale, thickness);
                cv2.rectangle(im, (row.x, row.y-10), (row.x+row.w, row.y), (0,0,0), -1);
                cv2.putText(im, label, (row.x, row.y), fontface, scale, (255,255,255), thickness, 8);            

                # put ID
                #label = str(row.id)
                #text = cv2.getTextSize(label, fontface, scale, thickness);
                #cv2.rectangle(im, (row.x, row.y+row.height), (row.x+row.w, row.y+row.height+10), (0,0,0), -1);
                #cv2.putText(im, label, (row.x, row.y+row.height+10), fontface, scale, (255,255,255), thickness, 8);  

                # draw bounding box
                cv2.rectangle(im, (row.x, row.y), (row.x+row.w, row.y+row.h), color=(0, 0, 255), thickness=2)

        op_cap.write(im)
        t+=1
        #if(t==100):
        #    break

    in_cap.release()
    op_cap.release() 
    # ------- main ------- #

Drawing trajectories



 54%|███████████████████████████████████████████▍                                    | 114/210 [00:10<00:08, 10.94it/s]


100%|████████████████████████████████████████████████████████████████████████████████| 210/210 [00:17<00:00, 15.33it/s]

Drawing trajectories




  0%|                                                                                          | 0/180 [00:00<?, ?it/s]

  1%|▉                                                                                 | 2/180 [00:00<00:11, 15.31it/s]

  2%|█▊                                                                                | 4/180 [00:00<00:11, 14.83it/s]

  3%|██▋                                                                               | 6/180 [00:00<00:11, 15.01it/s]

  4%|███▋                                                                              | 8/180 [00:00<00:12, 13.40it/s]

  6%|████▌                                                                            | 10/180 [00:00<00:13, 12.80it/s]

  7%|█████▍                                                                           | 12/180 [00:00<00:14, 11.49it/s]

  8%|██████▎                                                                          | 14/180 [00:01<00:16,  9.90it/s]

  8%|██████▊                  

...










































  0%|                                                                                          | 0/147 [00:00<?, ?it/s]








































  1%|█                                                                                 | 2/147 [00:00<00:11, 13.13it/s]








































  3%|██▏                                                                               | 4/147 [00:00<00:10, 13.59it/s]








































  4%|███▎                                                                              | 6/147 [00:00<00:10, 13.50it/s]








































  5%|████▍                                                                             | 8/147 [00:00<00:10, 13.61it/s]








































  7%|█████▌                                                                           | 10/147 [00:00<00:10, 13.39it/s]









































 70%|████████████████████████████████████████████████████████                        | 103/147 [00:07<00:02, 16.00it/s]








































 71%|█████████████████████████████████████████████████████████▏                      | 105/147 [00:07<00:02, 15.79it/s]








































 73%|██████████████████████████████████████████████████████████▏                     | 107/147 [00:07<00:02, 15.92it/s]








































 74%|███████████████████████████████████████████████████████████▎                    | 109/147 [00:07<00:02, 15.35it/s]








































 76%|████████████████████████████████████████████████████████████▍                   | 111/147 [00:07<00:02, 15.18it/s]








































 77%|█████████████████████████████████████████████████████████████▍                  | 113/147 [00:07<00:02, 15.19it/s]








































 78%|███████████████████████████████████

Drawing trajectories


ValueError: cannot convert float NaN to integer









































301it [00:32, 14.82it/s]








































151it [00:22, 14.23it/s]

In [ ]:
args = AttrDict(checkpoint['args'])  
classifier = CNNLSTM2(
    embedding_dim=args.embedding_dim,
    h_dim=args.h_dim,
    dropout=0.0,
    grad=True)

In [ ]:
checkpoint = torch.load("models/cnnlstm2-vgg16-lausanne-noembedder_with_model.pt", map_location='cpu')
print(checkpoint['best_state'])

In [ ]:
for key, value in checkpoint.items() :
    print(key)
    
print(checkpoint['best_t'])
for key, value in checkpoint['best_state'].items() :
    if(key == "lstm.weight_hh_l0"):
        a = value
        
for name, param in classifier.named_parameters():
    if(name == "lstm.weight_hh_l0"):
        b = param
for name, param in classifier.named_parameters():
    print(name)
print(np.linalg.norm((a-b).detach().numpy()))

In [ ]:
for p in classifier.named_children():
    #if(p == 'lstm'):
    print(p)

In [ ]:
print(checkpoint['metrics_val']['d_precision'][57])

In [ ]:
import os
import torch
import numpy as np
import cv2
import matplotlib.pyplot as plt

checkpoint = torch.load("./models/ckpt_with_model.pt", map_location='cpu')
plt.figure()
plt.title("loss")
plt.plot(checkpoint['metrics_train']['d_loss'], 'r', checkpoint['metrics_val']['d_loss'], 'b')
plt.legend(["train", "val"])
plt.figure()
plt.title("precision")
plt.plot(checkpoint['metrics_train']['d_precision'], 'r', checkpoint['metrics_val']['d_precision'], 'b')
plt.legend(["train", "val"])
plt.figure()
plt.title("recall")
plt.plot(checkpoint['metrics_train']['d_recall'], 'r', checkpoint['metrics_val']['d_recall'], 'b')
plt.legend(["train", "val"])

print(che)

In [ ]:
checkpoint = torch.load("./vgg16-lausanne.pt", map_location='cpu')
print(checkpoint['best_state'])

In [ ]:
checkpoint = torch.load("./vgg16-lausanne.pt", map_location='cpu')
print(checkpoint['best_state'])
plt.figure()
plt.title("loss")
plt.plot(checkpoint['metrics_train']['d_loss'], 'r', checkpoint['metrics_val']['d_loss'], 'b')
plt.legend(["train", "val"])
plt.figure()
plt.title("precision")
plt.plot(checkpoint['metrics_train']['d_precision'], 'r', checkpoint['metrics_val']['d_precision'], 'b')
plt.legend(["train", "val"])
plt.figure()
plt.title("recall")
plt.plot(checkpoint['metrics_train']['d_recall'], 'r', checkpoint['metrics_val']['d_recall'], 'b')
plt.legend(["train", "val"])